In [160]:
import numpy as np

# Class of State 

In [161]:
class State:
    def __init__(self, vector):
        self.vector = vector

    def apply(self, gate):
        return State(gate.matrix@self.vector)

    def norm(self):
        return np.linalg.norm(self.vector)
    
    def normalise(self):
        return State(self.vector/self.norm())
    
    def __str__(self):
        return str(self.vector)
    
    def __pow__(self, other):
        if isinstance(self, State):
            return State(np.kron(self.vector,other.vector))
    
    def full_measurement(state):
        # Calculate probabilities for each basis state
        probs = np.abs(state)**2

        # Randomly select a basis state based on probabilities
        outcome_index = np.random.choice(len(state), p=probs)

        # Collapse the state to the selected basis state
        collapsed_state = np.zeros_like(state)
        collapsed_state[outcome_index] = 1

        return outcome_index, collapsed_state
    
    def subsystem_measurement(self, qubit_index, num_qubits):
        """
        Perform measurement on a specific qubit in a multi-qubit system and remove the measured dimension.
        """
        # Reshape the state vector into a tensor product form
        reshaped_state = self.vector.reshape([2] * num_qubits)

        # Calculate probabilities for the specified qubit
        probs = np.zeros(2)
        for i in range(2):
            mask = [slice(None)] * num_qubits
            mask[qubit_index] = i
            probs[i] = np.sum(np.abs(reshaped_state[tuple(mask)])**2)

        # Randomly select a measurement outcome
        outcome = np.random.choice([0, 1], p=probs)

        # Collapse the state based on the measurement outcome
        mask = [slice(None)] * num_qubits
        mask[qubit_index] = outcome
        collapsed_state = reshaped_state[tuple(mask)]

        # Remove the measured dimension
        final_state = State(collapsed_state.flatten())
        new_state_vector = final_state.normalise()

        return outcome, new_state_vector
    
    def __str__(self):
        return str(self.vector.round(4))

# Class of Gate

In [162]:
class Gate:
    def __init__(self, matrix):
        self.matrix = matrix
    
    def __matmul__(self, other):
        if isinstance(other, Gate):
            return Gate(other.matrix@self.matrix)
        elif isinstance(other, State):
            return other.apply(self)
        else:
            raise TypeError("Unsupported type for multiplication with Gate")
        
    def __add__(self, other):
        if isinstance(other, Gate):
            return Gate(self.matrix + other.matrix)
        else:
            raise TypeError("Unsupported type for addition with Gate")
    
    def __pow__(self, other):
        return Gate(np.kron(self.matrix, other.matrix))

# Useful States and Gates

In [ ]:
zero_state = State(np.array([1, 0]))  # |0>
one_state  = State(np.array([0, 1]))  # |1>

phi_plus  = State(np.array([1, 0, 0, 1]) / np.sqrt(2))  # |00> + |11>
phi_minus = State(np.array([1, 0, 0, -1]) / np.sqrt(2)) # |00> - |11>
psi_plus  = State(np.array([0, 1, 1, 0]) / np.sqrt(2))  # |01> + |10>
psi_minus = State(np.array([0, 1, -1, 0]) / np.sqrt(2)) # |01> - |10>


I = Gate(np.array([[1, 0], [0, 1]]))
X = Gate(np.array([[0, 1], [1, 0]]))    # Pauli-X gate
Y = Gate(np.array([[0, -1j], [1j, 0]]))  # Pauli-Y gate
Z = Gate(np.array([[1, 0], [0, -1]]))   # Pauli-Z gate
H = Gate((1/np.sqrt(2)) * np.array([[1, 1], [1, -1]]))  # Hadamard gate
T = Gate(np.array([[1, 0], [0, np.exp(1j * np.pi / 4)]]))  # T gate
CNOT = Gate(np.array([[1, 0, 0, 0], 
                 [0, 1, 0, 0], 
                 [0, 0, 0, 1], 
                 [0, 0, 1, 0]]))  # CNOT gate
project_zero = Gate(np.array([[1, 0], [0, 0]]))  # Projector onto |0>
project_one  = Gate(np.array([[0, 0], [0, 1]]))  # Projector onto |1>

# One-qubit Gate: H@T@X Gate

In [164]:
# Example 1: H@T@X gate

initial_state = one_state
full_gate = H@T@X
final_state = full_gate@initial_state

print("HTX gate")
print("Final state:", final_state)
print("Final state norm:", final_state.norm())

HTX gate
Final state: [-0.5   -0.5j  0.7071+0.j ]
Final state norm: 0.9999999999999999


# Two-qubit Gate: H+CNOT gate

In [165]:
# Example 2: parallel H's and CNOT gate

initial_state = one_state**one_state
full_gate = CNOT@(H**H)
final_state = full_gate@initial_state

print()
print("Final state:", final_state)
print("Final state norm:", final_state.norm())


Final state: [ 0.5  0.5 -0.5 -0.5]
Final state norm: 0.9999999999999998


# Three-qubit Gate: Quantum Teleportation

In [166]:
# Example 3: Quantum teleportation

alpha, beta = 0.3**0.5, 0.7**0.5

chris_state = State(np.array([alpha,beta]))

print("initial state",chris_state)
print("norm",state.norm())

state = chris_state**phi_plus

state = (CNOT**I)@state
state = (H**I**I)@state

bell_measure_outcome = [None,None]
outcome, state = state.subsystem_measurement(0, 3)

bell_measure_outcome[0] = int(outcome)
outcome, state = state.subsystem_measurement(0, 2)

bell_measure_outcome[1] = int(outcome)

print()
print("state after bell measurment", state)
print("norm:", state.norm())
print("measurment_outcome", bell_measure_outcome)

final_gate = I
if bell_measure_outcome[0]: 
    final_gate = final_gate@Z
if bell_measure_outcome[1]: 
    final_gate = final_gate@X

final_state = final_gate@state
print()
print("final state", final_state)
print("norm:", final_state.norm())

initial state [0.5477 0.8367]
norm 1.0

state after bell measurment [0.5477 0.8367]
norm: 1.0
measurment_outcome [0, 0]

final state [0.5477 0.8367]
norm: 1.0


# Three Qubit Gate: Quantum Fourier Transform

In [171]:
#initial_state = zero_state ** zero_state ** zero_state
#initial_state = zero_state ** one_state ** one_state
initial_state = one_state ** zero_state ** zero_state
#initial_state = one_state ** one_state ** one_state


print("initial state",initial_state)
print("norm",state.norm())

def R(k):
    omega = 1j*2*np.pi
    Rk =[[1, 0],
         [0, np.exp(omega/2**k)]]
    return Gate(np.array(Rk))

gate = H**I**I
gate @= R(2)**project_zero**I + I**project_one**I
gate @= R(3)**I**project_zero + I**I**project_one

gate @= I**H**I
gate @= I**R(2)**project_zero + I**I**project_one

gate @= I**I**H

final_state = gate@initial_state

print()
print("final state", final_state.vector.round(3))
print("norm:", final_state.norm())

initial state [0 0 0 0 1 0 0 0]
norm 1.0

final state [0.354+0.j    0.354+0.j    0.   +0.354j 0.   +0.354j 0.25 -0.25j
 0.25 -0.25j  0.25 +0.25j  0.25 +0.25j ]
norm: 0.9999999999999997


In [168]:
print(1/np.sqrt(8))

0.35355339059327373
